# Fit the Model

In [ ]:
%%sh
drum fit --code-dir ./model \
--input data/10K_Lending_Club_Loans_utf-8.csv \
--output ./model \
--target-type binary \
--target is_bad \
--docker env 
--verbose
--logging-level debug

# Testing the model

## Performance Testing

In [ ]:
%%sh
drum perf-test \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env \
--verbose

## Validation

In [ ]:
%%sh
drum validation \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env > validation.log

In [ ]:
! tail -n 30 validation.log

## Batch Scoring

In [ ]:
%%sh
drum score \
--code-dir ./model \
--target-type binary \
--positive-class-label pos_class_prediction \
--negative-class-label neg_class_prediction \
--input data/10K_LC_short.csv \
--logging-level info \
--docker env \
--output data/predictions.csv

In [ ]:
!head -n 5 data/predictions.csv

## Batch Scoring Unstructured Model

Returns Predictions and SHAP Values

In [ ]:
%%sh
drum score \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type unstructured \
--content-type application/text \
--docker env \
--verbose --output data/predictions.json

In [ ]:
import json
import pprint
d = json.load(open("data/predictions.json", "rb"))
pprint.pprint(d["data"][0])

## Serving Model

In [ ]:
import subprocess
import requests
import time

In [ ]:
run_inference_server = [
    "drum", "server",
    "--code-dir","./model", 
    "--address", "0.0.0.0:6789", 
    "--show-perf",
    "--target-type", "binary",
    "--logging-level", "info",
    "--show-stacktrace",
    "--verbose", 
    "--docker", "env"
]
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [ ]:
## snoozing before pinging the server to give it time to actually start
time.sleep(5)
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

In [ ]:
## shutdown
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

In [ ]:
inference_server.terminate()
inference_server.stdout.readlines()

## Push Artifacts and Environment to DataRobot

Making a copy of the model folder

In [15]:
%%sh
rm -R ./model-c && rm -R ./model-ext
cp -R ./model ./model-c
cp -R ./model ./model-ext

In [16]:
!python ./utils/push.py --env-dir ./env --model-dir ./model-c --logging-level INFO --max-wait 600 --update-env False

Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model-c', update_env=False)
__main__ - INFO - 2021-01-07 20:28:48,172 - Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model-c', update_env=False)
__main__ - INFO - 2021-01-07 20:28:48,597 - load model config
__main__ - INFO - 2021-01-07 20:28:48,604 - {'description': None, 'language': 'Python', 'majorVersion': True, 'name': 'lgbm-classifier', 'negativeClassLabel': 0, 'positiveClassLabel': 1, 'predictionThreshold': 0.17, 'targetName': 'is_bad', 'targetType': 'binary', 'testData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv', 'trainingData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_Lending_Club_Loans_utf-8.csv', 'type': 'inference', 'versionId': None, 'environmentID': '5ff79aa56161d5e27deb4628', 'environmentVersionID': '5ff79aa6e85ab3dbce50543b'}
__main__ - INFO - 2021-01-07 20:28:48,604 - create new inference model
__main__ - INFO - 2021-0

# Test Model in DataRobot

In [17]:
!python ./utils/test.py --model-dir ./model-c --logging-level INFO --max-wait 600

Namespace(logging_level='INFO', max_wait=600, model_dir='./model-c')
__main__ - INFO - 2021-01-07 20:31:04,649 - Namespace(logging_level='INFO', max_wait=600, model_dir='./model-c')
__main__ - INFO - 2021-01-07 20:31:04,649 - loading model config
__main__ - INFO - 2021-01-07 20:31:04,657 - loading dataset to ai catalog
__main__ - INFO - 2021-01-07 20:31:41,860 - starting custom model test
__main__ - INFO - 2021-01-07 20:36:16,690 - test complete
__main__ - WARNING - 2021-01-07 20:36:16,690 - warning 😱
__main__ - WARNING - 2021-01-07 20:36:16,690 - Test: error_check
__main__ - WARNING - 2021-01-07 20:36:16,690 - Status: succeeded
__main__ - WARNING - 2021-01-07 20:36:16,690 - Message: 
__main__ - WARNING - 2021-01-07 20:36:16,690 - Test: null_value_imputation
__main__ - WARNING - 2021-01-07 20:36:16,690 - Status: warning
__main__ - WARNING - 2021-01-07 20:36:16,690 - Message: Model cannot impute null values for the following columns:
int_rate, zip_code, addr_state
__main__ - WARNING - 2

# Deploy Model to DataRobot

In [19]:
!python ./utils/deploy.py --model-dir ./model-c --external False

Namespace(external=False, logging_level='INFO', max_wait=600, mlops_api_token='L-KEWy2boF5520IYONK096R-XzHOuVsk', mlops_endpoint='https://app.datarobot.com', model_dir='./model-c')
__main__ - INFO - 2021-01-07 20:45:07,144 - Namespace(external=False, logging_level='INFO', max_wait=600, mlops_api_token='L-KEWy2boF5520IYONK096R-XzHOuVsk', mlops_endpoint='https://app.datarobot.com', model_dir='./model-c')
__main__ - INFO - 2021-01-07 20:45:07,144 - deploying custom inference model
__main__ - INFO - 2021-01-07 20:45:07,469 - connected https://app.datarobot.com/api/v2
__main__ - INFO - 2021-01-07 20:45:07,470 - load model config for custom model deployment
__main__ - INFO - 2021-01-07 20:45:07,477 - {'datasets': {'testDatasetID': '5ff7b5d839626b4f255052a5',
              'trainingDataCatalogId': '5ff7b5517d4c9428159143b5'},
 'description': None,
 'environmentID': '5ff79aa56161d5e27deb4628',
 'environmentVersionID': '5ff79aa6e85ab3dbce50543b',
 'id': '5ff7b55015553bd185505431',
 'language': 

# Deploy Model outside of DataRobot with DR Tracking Agents

In [ ]:
# !python ./utils/deploy.py --model-dir ./model-ext --external True